# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import CSVFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [27]:
text_loader = CSVFileLoader("data/at_map_points.csv")
documents = text_loader.load_documents()
len(documents)

['Location: Max Epperson Shelter.\nDescription: April 8 shelter. Night 0.', "Location: Black Gap Shelter\nDescription: Night 1. First night in tent. No condensation, yay. Olivier's sleeping pad didn't stay inflated, boo. Freezing temps all night, but got some sleep.", 'Location: Lunch\nDescription: Some small hail. Sub 40 degrees all day. 10/10 score from Olivier for the pulled pork packets.', 'Location: Start of Appalachian Trail\nDescription: Southern terminus of the AT. Nice family took our photo.', "Location: Lunch\nDescription: Olivier's left knee hurts.", "Location: Hawk Mountain Shelter\nDescription: Night 2. Hawk Mountain Shelter. Olivier's knee got better over time this afternoon. Many more people than last night. Traded brownie for broccoli to add to our dinner. Hoping the dog next door doesn't bark, and that the old man that just got to camp shuts up.", 'Location: Trail Magic\nDescription: Eggs, sausage, coffee, and fruit from a nice man outside of his camper at Hightower Ga

In [26]:
from aimakerspace.text_utils import CSVFileLoader
loader = CSVFileLoader("data/at_map_points.csv")
documents = loader.load_documents()
print(f"Loaded {len(documents)} documents")
print("First document:")
print(documents[0])

Loaded 474 documents
First document:
Location: Max Epperson Shelter.
Description: April 8 shelter. Night 0.


In [23]:
print(documents)

['Location: Max Epperson Shelter.\nDescription: April 8 shelter. Night 0.', "Location: Black Gap Shelter\nDescription: Night 1. First night in tent. No condensation, yay. Olivier's sleeping pad didn't stay inflated, boo. Freezing temps all night, but got some sleep.", 'Location: Lunch\nDescription: Some small hail. Sub 40 degrees all day. 10/10 score from Olivier for the pulled pork packets.', 'Location: Start of Appalachian Trail\nDescription: Southern terminus of the AT. Nice family took our photo.', "Location: Lunch\nDescription: Olivier's left knee hurts.", "Location: Hawk Mountain Shelter\nDescription: Night 2. Hawk Mountain Shelter. Olivier's knee got better over time this afternoon. Many more people than last night. Traded brownie for broccoli to add to our dinner. Hoping the dog next door doesn't bark, and that the old man that just got to camp shuts up.", 'Location: Trail Magic\nDescription: Eggs, sausage, coffee, and fruit from a nice man outside of his camper at Hightower Ga

### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [8]:
# 💡 For my updates, my personal data is already chunked,
# so this code is no longer needed.
# text_splitter = CharacterTextSplitter()
# split_documents = text_splitter.split_texts(documents)
# len(split_documents)

# Maintain variable names to simplify following updates
split_documents = documents

Let's take a look at some of the documents we've managed to split.

In [10]:
split_documents[0:3]

['Location: Max Epperson Shelter.\nDescription: April 8 shelter. Night 0.',
 "Location: Black Gap Shelter\nDescription: Night 1. First night in tent. No condensation, yay. Olivier's sleeping pad didn't stay inflated, boo. Freezing temps all night, but got some sleep.",
 'Location: Lunch\nDescription: Some small hail. Sub 40 degrees all day. 10/10 score from Olivier for the pulled pork packets.']

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

##### ✅ Answer:

1. Yes, we can pass in the optional `dimensions` parameter to set a specific dimensionality.
    
    It was impossible to find information about if there's a limit on what dimensions you can pass, so I tried it out. Turns out you can't go larger than the default of 1536 (duh, I guess) and you do less than 1. (Also duh, but you gotta try it, right?) but what's interesting is that within that range you can choose any number for the dimensions, while asking an LLM tries to convince you that only specific numbers are allowed, likely because external libraries only support specific embedding sizes.

2. Matryoshka Representation Learning (MRL) is the concept that allows us to essentially "chop off" the ends of embeddings and still maintain semantic meaning.

    At a high level, this means that the further into our embedding we look, the more granular it will be in its representation. In reality, we have to do some normalization, we can't truly just slice off the end of an embedding, but this is a really cool feature considering that it used to be if you cut off part of an embedding, it lost its meaning entirely.

##### ✅ End Answer

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:

The benefit of using asynchronous code to create our embeddings is that we can make many concurrent calls to the embedding model at once, which explains why the above cell ran so quickly. Simply using `asyncio` isn't magic, though. We can follow the chain of method calls from `abuild_from_list` to `async_get_embeddings` where we finally see that we're calling our embedding model in batches, processing multiple batches at once using `asyncio.gather` (similar to `Promise.all()`). Neat!

One might be tempted to think, why don't we run smaller batches concurrently to speed it up even more! This would be an interesting idea, but the batch processing done by OpenAI is more efficient than single requests or smaller batches. There might be some room for optimization, but generally speaking using OpenAI's native batch processing is a better call.

##### ✅ End Answer

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:

Hands down, the best way to create more reproducible results is going to be through **prompt engineering**, and more specifically **few-shot prompting**.

Providing specific instructions on how to behave, what tone to use, how specific to be, etc., is a good start to getting consistent results.

Even with specific instructions, just like with humans, they are up to interpretation. The best way to get a human do to do something the way you want them to is to show them how to do it. It's the same with an LLM. If we give several consistent examples of what we want the LLM to do, it's going to be very likely to consistently follow those examples, given it's provided with a prompt that's appropriate to those examples.

You could also consider things like top-p and temperature, but that's going to have unintended consequences unless you're goal is for your model to cut down on creativity and unique results. Either way, this should still be on top of few-shot prompting if the goal is true consistency.

##### ✅ End Answer

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

The best way to write a loop in Python often depends on the specific task you're trying to accomplish and the type of data you're working with. Here are a few general guidelines to help you write effective loops:

1. **Choose the Right Loop Structure**:
   - Use a `for` loop when iterating over a collection (like a list, tuple, dictionary, or string).
   - Use a `while` loop when you need to repeat a block of code until a certain condition is met.

2. **Keep It Simple**:
   - Write clear and concise loop conditions to avoid confusion.
   - Limit the complexity of operations inside the loop to enhance readability.

3. **Use `enumerate()` for Indexing**:
   - If you need both the index and the value from an iterable, use `enumerate()`. It provides the index and value as you iterate.
     ```python
     my_list = ['a', 'b', 'c']
     for index, value in enumerate(my_list):
         print(index, value)
     ```

4. **List Comprehensions for Simple Loops**:
   - For simple loops that genera

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [12]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [13]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [17]:
print(system_kwargs)

NameError: name 'system_kwargs' is not defined

In [14]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: The 'Michael Eisner Memorial Weak Executive Problem' refers to a situation where a CEO, who has a strong background in a particular area (such as product management, sales, or marketing), hires a weak executive for that same function. This phenomenon often occurs because the CEO struggles to let go of the expertise that initially contributed to their success and, as a result, selects a less competent executive in that role to maintain a sense of control and remain “the man” in that area. This issue is exemplified by Michael Eisner, the former CEO of Disney, who, despite being a skilled television network executive, faced challenges when ABC fell to fourth place after its acquisition. His response, indicating that he could turn around the network if he had more time, highlights the unrealistic expectations that can stem from this problem.

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

##### ✅ Answer:

The first thing I checked is the prompt we're already sending. We already have our `response_length` set to "comprehensive" and `response_style` set to "detailed". That means we have an instruction in our system message of:

"Keep responses detailed and comprehensive"

Given this, I would probably start with two approachs:

1. Adding further instructions about what we mean by "comprehensive." Is this multiple paragraphs? An overview followed by a breakdown of key points and a real-world example?

2. Provide few-shot prompting. If we're looking to produce the same style of explanations over and over, we would benefit from showing several examples in addition to our more complete instructions.

The strategies I've discussed broadly fall under "prompt engineering," but more precisely I think "few-shot prompting" would have the greatest impact.

##### ✅ End Answer

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

##### ✅ Start Activity #1

##### Activity #1 Planning:

- Allow it to work with PDF files
    Maybe this means ingesting PDF files, maybe it means writing our output to a PDF file. Either way, file management doesn't sound like the most interesting use of time.

- Implement a new distance metric
    This could be interesting to see if we get any different results, but given the size of our dataset (rather small) I don't think it would make a big difference.

- ⭐️ Add metadata support to the vector database
    ~~This seems to be the most interesting suggestion, but it also comes with a challenge because the `PMarcaBlogs.txt` files is very unstructured. There's nothing cohesive enough here to create metadata from. It would be difficult to parse out titles or separators to determine what chunks belong to which blog.~~

    ~~This would allow us to reference the blogs in the output and allow the user to dig deeper if they want to. Maybe we can even provide URLs, but we'll need to see what format we're getting the blogs in.~~

    Given that the blogs aren't provided in a nice format and the online source seems to be only available in HTML, I'll update with my own structured data to allow for this implementation.

    1. Add personal blog entries
    2. Update document parsing to handle new format (CSV)
    3. Update to store metadata (location, title of blog post)
    4. Update prompts to be appropriate for new personal data

In [18]:
### YOUR CODE HERE